In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from os.path import exists
from os import remove
import json
from sklearn.model_selection import train_test_split
import sklearn.model_selection as ms
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

import sys
sys.path.insert(0, "../util/")
import util as util

%autosave 5

Autosaving every 5 seconds


# Adding dummies and train-test-split

In [8]:
maleTrainWithDummiesExists = exists('../data/male/maleTrainWithDummies.csv')
maleTestWithDummiesExists = exists('../data/male/maleTestWithDummies.csv')
maleTrainWithoutDummiesExists = exists('../data/male/maleTrainWithoutDummies.csv')
maleTestWithoutDummiesExists = exists('../data/male/maleTestWithoutDummies.csv')
maleFullWithDummiesExists = exists('../data/male/maleFullWithDummies.csv')
maleFullWithoutDummiesExists = exists('../data/male/maleFullWithoutDummies.csv')

femaleTrainWithDummiesExists = exists('../data/female/femaleTrainWithDummies.csv')
femaleTestWithDummiesExists = exists('../data/female/femaleTestWithDummies.csv')
femaleTrainWithoutDummiesExists = exists('../data/female/femaleTrainWithoutDummies.csv')
femaleTestWithoutDummiesExists = exists('../data/female/femaleTestWithoutDummies.csv')
femaleFullWithDummiesExists = exists('../data/female/femaleFullWithDummies.csv')
femaleFullWithoutDummiesExists = exists('../data/female/femaleFullWithoutDummies.csv')

relatedDummiesDictionaryExists = exists('../data/relatedDummiesDictionary.json')

In [9]:
with open('../data/columnDataDictionary.json') as d:
    columnDataDictionary = json.load(d)
columnList = columnDataDictionary['columnList']
nonBinaryCategoricalList = columnDataDictionary['nonBinaryCategoricalList']
stringToFloatList = columnDataDictionary['stringToFloatList']
pointDistributionList = columnDataDictionary['pointDistributionList']
partnerList = columnDataDictionary['partnerList']

In [10]:
if (maleTrainWithDummiesExists and maleTestWithDummiesExists and maleFullWithDummiesExists and
    maleTrainWithoutDummiesExists and maleTestWithoutDummiesExists and maleFullWithoutDummiesExists and
    femaleTrainWithDummiesExists and femaleTestWithDummiesExists and femaleFullWithDummiesExists and
    femaleTrainWithoutDummiesExists and femaleTestWithoutDummiesExists and femaleFullWithoutDummiesExists and
    relatedDummiesDictionaryExists):
    maleTrainWithDummies = pd.read_csv('../data/male/maleTrainWithDummies.csv')
    maleTestWithDummies = pd.read_csv('../data/male/maleTestWithDummies.csv')
    maleTrainWithoutDummies = pd.read_csv('../data/male/maleTrainWithoutDummies.csv')
    maleTestWithoutDummies = pd.read_csv('../data/male/maleTestWithoutDummies.csv')
    maleFullWithDummies = pd.read_csv('../data/male/maleFullWithDummies.csv')
    maleFullWithoutDummies = pd.read_csv('../data/male/maleFullWithoutDummies.csv')
    
    femaleTrainWithDummies = pd.read_csv('../data/female/femaleTrainWithDummies.csv')
    femaleTestWithDummies = pd.read_csv('../data/female/femaleTestWithDummies.csv')
    femaleTrainWithoutDummies = pd.read_csv('../data/female/femaleTrainWithoutDummies.csv')
    femaleTestWithoutDummies = pd.read_csv('../data/female/femaleTestWithoutDummies.csv')
    femaleFullWithDummies = pd.read_csv('../data/female/femaleFullWithDummies.csv')
    femaleFullWithoutDummies = pd.read_csv('../data/female/femaleFullWithoutDummies.csv')
    
    with open('../data/relatedDummiesDictionary.json') as d:
        relatedDummiesDictionary = json.load(d)
    for df in [maleTrainWithDummies,maleTestWithDummies,maleFullWithDummies,maleTrainWithoutDummies,maleTestWithoutDummies,maleFullWithoutDummies,
               femaleTrainWithDummies,femaleTestWithDummies,femaleFullWithDummies,femaleTrainWithoutDummies,femaleTestWithoutDummies,femaleFullWithoutDummies]:
        for col in nonBinaryCategoricalList:
            df[col] = df[col].apply(str)
else:
    !rm -r ../data/male
    !mkdir ../data/male
    !rm -r ../data/female
    !mkdir ../data/female
    
    !jupyter nbconvert --execute eda.ipynb
    
    datingData = pd.read_csv('../data/encoded-SpeedDatingData.csv')
    blindDateData = datingData[columnList]
    for col in nonBinaryCategoricalList:
        blindDateData[col] = blindDateData[col].apply(str)
    for col in stringToFloatList:
        blindDateData[col] = blindDateData[col].str.replace(',', '').astype(float)
    blindDateData['zipcode'] = blindDateData[col].str.replace(',', '').astype(float)
    blindDateData['zipcode'] = blindDateData[col].apply(str)
    
    blindDateCategoricalData = blindDateData.select_dtypes(include=['O'])
    for col in blindDateCategoricalData.columns:
        blindDateData[col]=blindDateData[col].fillna('nan')
    relatedDummiesDictionary = {}
    for col in blindDateCategoricalData.columns:
        dummyData = pd.get_dummies(blindDateData[col],prefix=col,drop_first=True)
        for dummyCol in dummyData.columns:
            relatedDummiesDictionary[str(dummyCol)] = list(dummyData.columns)
            if col in partnerList:
                partnerList.append(str(dummyCol))
        blindDateData = pd.concat([blindDateData,dummyData],axis=1)
    with open('../data/relatedDummiesDictionary.json', 'w') as fp:
        json.dump(relatedDummiesDictionary, fp)
        
    partnerList = list(set(partnerList))
    columnDataDictionary = {"columnList": columnList,
                        "nonBinaryCategoricalList": nonBinaryCategoricalList,
                        "stringToFloatList": stringToFloatList,
                        "pointDistributionList": pointDistributionList,
                        "partnerList": partnerList}

    with open('../data/columnDataDictionary.json', 'w') as fp:
            json.dump(columnDataDictionary, fp)
    
    maleFullWithDummies = blindDateData[blindDateData['gender'] == 1]
    femaleFullWithDummies = blindDateData[blindDateData['gender'] == 0]
    
    femaleChoice = maleFullWithDummies['dec_o']
    XMale = maleFullWithDummies.drop(['dec_o'], axis=1)
    maleChoice = femaleFullWithDummies['dec_o']
    XFemale = femaleFullWithDummies.drop(['dec_o'], axis=1)
    
    maleTrainWithDummies, maleTestWithDummies, femaleChoiceTrain, femaleChoiceTest = train_test_split(XMale, femaleChoice, test_size=0.2)
    femaleTrainWithDummies, femaleTestWithDummies, maleChoiceTrain, maleChoiceTest = train_test_split(XFemale, maleChoice, test_size=0.2)
    
    maleTrainWithDummies['dec_o'] = femaleChoiceTrain
    maleTestWithDummies['dec_o'] = femaleChoiceTest
    femaleTrainWithDummies['dec_o'] = maleChoiceTrain
    femaleTestWithDummies['dec_o'] = maleChoiceTest
    
    maleTrainWithDummies.to_csv('../data/male/maleTrainWithDummies.csv',index=False)
    maleTestWithDummies.to_csv('../data/male/maleTestWithDummies.csv',index=False)
    maleFullWithDummies.to_csv('../data/male/maleFullWithDummies.csv',index=False)
    
    femaleTrainWithDummies.to_csv('../data/female/femaleTrainWithDummies.csv',index=False)
    femaleTestWithDummies.to_csv('../data/female/femaleTestWithDummies.csv',index=False)
    femaleFullWithDummies.to_csv('../data/female/femaleFullWithDummies.csv',index=False)
    
    dummyColumns = list(relatedDummiesDictionary.keys())
    maleTrainWithoutDummies = maleTrainWithDummies.drop(dummyColumns, axis=1)
    maleTestWithoutDummies = maleTestWithDummies.drop(dummyColumns, axis=1)
    maleFullWithoutDummies = maleFullWithDummies.drop(dummyColumns, axis=1)
    femaleTrainWithoutDummies = femaleTrainWithDummies.drop(dummyColumns, axis=1)
    femaleTestWithoutDummies = femaleTestWithDummies.drop(dummyColumns, axis=1)
    femaleFullWithoutDummies = femaleFullWithDummies.drop(dummyColumns, axis=1)
    
    maleTrainWithoutDummies.to_csv('../data/male/maleTrainWithoutDummies.csv',index=False)
    maleTestWithoutDummies.to_csv('../data/male/maleTestWithoutDummies.csv',index=False)
    maleFullWithoutDummies.to_csv('../data/male/maleFullWithoutDummies.csv',index=False)
    femaleTrainWithoutDummies.to_csv('../data/female/femaleTrainWithoutDummies.csv',index=False)
    femaleTestWithoutDummies.to_csv('../data/female/femaleTestWithoutDummies.csv',index=False)
    femaleFullWithoutDummies.to_csv('../data/female/femaleFullWithoutDummies.csv',index=False)

/Users/garysimmons/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (27) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


## Data Cleaning

### Female

#### Join partner

In [11]:
for partnerCol in partnerList:
    if ((partnerCol+"_o" not in femaleTrainWithoutDummies.columns) and (partnerCol+"_o" not in femaleTrainWithDummies.columns)):
        femaleTrainWithDummies = util.joinToPartner(femaleTrainWithDummies,maleFullWithDummies)
        femaleTrainWithoutDummies = util.joinToPartner(femaleTrainWithoutDummies,maleFullWithoutDummies)
        femaleTrainWithDummies.to_csv('../data/female/femaleTrainWithDummies.csv',index=False)
        femaleTrainWithoutDummies.to_csv('../data/female/femaleTrainWithoutDummies.csv',index=False)
        break

#### Get distance

In [12]:
if (("partnerDistance" not in femaleTrainWithoutDummies.columns) or 
    ("partnerDistance" not in femaleTrainWithDummies.columns)):
    femaleTrainWithoutDummies = util.returnDFWithpartnerDistance(femaleTrainWithoutDummies,True)
    femaleTrainWithDummies["partnerDistance"] = femaleTrainWithoutDummies["partnerDistance"]
    femaleTrainWithDummies.to_csv('../data/female/femaleTrainWithDummies.csv',index=False)
    femaleTrainWithoutDummies.to_csv('../data/female/femaleTrainWithoutDummies.csv',index=False)

TypeError: string indices must be integers

#### Fix ambiguous scores

#### Replace Nans

### Male

#### Join partner

#### Get distance

#### Fix ambiguous scores

#### Replace Nans

# Training

## Logistic Regression

## KNN

## Random Forest Classifier

## Boosting

# Individual Testing

## Logistic Regression

## KNN

## Random Forest Classifier

## Boosting

# Ensemble Testing